# [Data Preparing](https://github.com/CMACH508/DeepTrader?tab=readme-ov-file#data-preparing)

In [7]:
TWII = "^TWII"
# 2025-09-04 wiki updated
TWII_STOCKS = [
    "1101.TW", "1216.TW", "1301.TW", "1303.TW", "2002.TW", "2207.TW", "2301.TW", "2303.TW", "2308.TW", "2317.TW",
    "2327.TW", "2330.TW", "2345.TW", "2357.TW", "2379.TW", "2382.TW", "2383.TW", "2395.TW", "2412.TW", "2454.TW",
    "2603.TW", "2609.TW", "2615.TW", "2880.TW", "2881.TW", "2882.TW", "2883.TW", "2884.TW", "2885.TW", "2886.TW",
    "2887.TW", "2890.TW", "2891.TW", "2892.TW", "2912.TW", "3008.TW", "3017.TW", "3034.TW", "3045.TW", "3231.TW",
    "3661.TW", "3711.TW", "4904.TW", "4938.TW", "5871.TW", "5876.TW", "5880.TW", "6446.TW", "6505.TW", "6669.TW"
]
print(f"Total stocks to process: {len(TWII_STOCKS)}")

Total stocks to process: 50


### TWII大盤資料下載

In [8]:
import yfinance as yf
from curl_cffi import requests as curl_requests

# 創建一個模擬 Chrome 瀏覽器的 session
session = curl_requests.Session(impersonate="chrome")

In [9]:
TWII
twii_history = yf.download(TWII, start='2000-01-01', end='2025-08-31', progress=False, auto_adjust=False, session=session)
twii_history.columns = twii_history.columns.droplevel(level=1)
twii_history = twii_history[["Open", "High", "Low", "Close", "Adj Close", "Volume"]]
twii_history.to_csv(f"./^TWII.csv")
twii_history

Price,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2000-01-04,8644.910156,8803.610352,8642.500000,8756.549805,8756.509766,0
2000-01-05,8690.599609,8867.679688,8668.019531,8849.870117,8849.830078,0
2000-01-06,8900.559570,9023.990234,8833.910156,8922.030273,8921.989258,0
2000-01-07,8853.429688,8940.769531,8739.429688,8849.870117,8849.830078,0
2000-01-10,8941.929688,9126.019531,8891.790039,9102.599609,9102.557617,0
...,...,...,...,...,...,...
2025-08-25,24108.810547,24362.019531,24108.750000,24277.380859,24277.380859,3818800
2025-08-26,24237.359375,24382.419922,24113.029297,24305.099609,24305.099609,5104500
2025-08-27,24449.810547,24539.880859,24410.080078,24519.900391,24519.900391,4022200


In [10]:
twii_ticker = yf.Ticker(TWII)
twii_history = twii_ticker.history(start='2000-01-01', end='2025-08-31', auto_adjust=False)
twii_history

,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits
Date,,,,,,,,
2000-01-04 00:00:00+08:00,8644.910156,8803.610352,8642.500000,8756.549805,8756.509766,0,0.0,0.0
2000-01-05 00:00:00+08:00,8690.599609,8867.679688,8668.019531,8849.870117,8849.830078,0,0.0,0.0
2000-01-06 00:00:00+08:00,8900.559570,9023.990234,8833.910156,8922.030273,8921.989258,0,0.0,0.0
2000-01-07 00:00:00+08:00,8853.429688,8940.769531,8739.429688,8849.870117,8849.830078,0,0.0,0.0
2000-01-10 00:00:00+08:00,8941.929688,9126.019531,8891.790039,9102.599609,9102.557617,0,0.0,0.0
...,...,...,...,...,...,...,...,...
2025-08-25 00:00:00+08:00,24108.810547,24362.019531,24108.750000,24277.380859,24277.380859,3818800,0.0,0.0
2025-08-26 00:00:00+08:00,24237.359375,24382.419922,24113.029297,24305.099609,24305.099609,5104500,0.0,0.0
2025-08-27 00:00:00+08:00,24449.810547,24539.880859,24410.080078,24519.900391,24519.900391,4022200,0.0,0.0


### 黃金/美元TWDUSD=X資料下載

In [11]:
TWDUSD='TWDUSD=X'
twd_usd_history = yf.download(TWDUSD, start='2000-01-01', end='2025-08-31', progress=False, auto_adjust=False, session=session)
twd_usd_history.columns = twd_usd_history.columns.droplevel(level=1)
twd_usd_history = twd_usd_history[["Open", "High", "Low", "Close", "Adj Close", "Volume"]]
twd_usd_history.to_csv(f"./TWDUSD=X.csv")

In [12]:
# Get the earliest available date for the TWII index
twii_ticker = yf.Ticker(TWII, session=session)
twii_history = twii_ticker.history(period="max")
if not twii_history.empty:
    twii_earliest_date = twii_history.index.min()
    print("TWII earliest date:", twii_earliest_date)
else:
    print("No historical data found for TWII.")

# Get the earliest available date for each stock in TWII_STOCKS\
# Create a Tickers object for multiple stocks
twii_tickers = yf.Tickers(" ".join(TWII_STOCKS))

# Dictionary to hold each stock's earliest date
stocks_earliest_dates = {}

for stock in TWII_STOCKS:
    ticker = twii_tickers.tickers[stock]
    stock_history = ticker.history(period="max")
    if not stock_history.empty:
        earliest_date = stock_history.index.min()
        stocks_earliest_dates[stock] = earliest_date
    else:
        stocks_earliest_dates[stock] = None

print("TWII Stocks earliest dates:")
for stock, date in stocks_earliest_dates.items():
    print(f"{stock}: {date}")

TWII earliest date: 1997-07-02 00:00:00+08:00
TWII Stocks earliest dates:
1101.TW: 2000-01-04 00:00:00+08:00
1216.TW: 2000-01-04 00:00:00+08:00
1301.TW: 2000-01-04 00:00:00+08:00
1303.TW: 2000-01-04 00:00:00+08:00
2002.TW: 2000-01-04 00:00:00+08:00
2207.TW: 2000-01-04 00:00:00+08:00
2301.TW: 2000-01-04 00:00:00+08:00
2303.TW: 2000-01-04 00:00:00+08:00
2308.TW: 2000-01-04 00:00:00+08:00
2317.TW: 1993-01-05 00:00:00+08:00
2327.TW: 2000-01-04 00:00:00+08:00
2330.TW: 2000-01-04 00:00:00+08:00
2345.TW: 2000-01-04 00:00:00+08:00
2357.TW: 2000-01-04 00:00:00+08:00
2379.TW: 2000-01-04 00:00:00+08:00
2382.TW: 2000-01-04 00:00:00+08:00
2383.TW: 2000-01-04 00:00:00+08:00
2395.TW: 2000-01-04 00:00:00+08:00
2412.TW: 2000-11-15 00:00:00+08:00
2454.TW: 2001-07-23 00:00:00+08:00
2603.TW: 2000-01-04 00:00:00+08:00
2609.TW: 2000-01-04 00:00:00+08:00
2615.TW: 2000-01-04 00:00:00+08:00
2880.TW: 2001-12-20 00:00:00+08:00
2881.TW: 2001-12-20 00:00:00+08:00
2882.TW: 2000-01-04 00:00:00+08:00
2883.TW: 2001-12

In [13]:
import datetime

for i in range(2000, 2025):
	stocks_to_remove = []
	target_date = datetime.date(i, 1, 10)
	for stock, dt in stocks_earliest_dates.items():
		if dt is None or dt.replace(tzinfo=None).date() > target_date:
			stocks_to_remove.append(stock)
	# print("Stocks to remove:", stocks_to_remove)
	print(str(i) + " " + str(len(stocks_to_remove)))


2000 22
2001 21
2002 17
2003 9
2004 7
2005 7
2006 6
2007 6
2008 6
2009 6
2010 5
2011 5
2012 4
2013 3
2014 3
2015 1
2016 1
2017 1
2018 0
2019 0
2020 0
2021 0
2022 0
2023 0
2024 0


In [14]:
import datetime

stocks_to_remove = []
target_date = datetime.date(2015, 1, 4)
for stock, dt in stocks_earliest_dates.items():
	if dt is None or dt.replace(tzinfo=None).date() > target_date:
		stocks_to_remove.append(stock)
print("Stocks to remove:", stocks_to_remove)
print(len(stocks_to_remove))

for stock in stocks_to_remove:
	if stock in TWII_STOCKS:
		TWII_STOCKS.remove(stock)

Stocks to remove: ['6669.TW']
1


In [1]:
import pandas as pd

# Generate business days for the fixed date range
business_days = pd.bdate_range(start="2015-01-01", end="2025-08-31")
print("Total business days:", len(business_days))
print()

intervals = {
    "Training": ("2015/01/01", "2019/12/31"),
    "Validation": ("2020/01/01", "2022/12/31"),
    "Test": ("2023/01/01", "2025/08/31"),
}
for interval_name, (start_date, end_date) in intervals.items():
    interval_days = pd.bdate_range(start=start_date, end=end_date)
    start_idx = business_days.get_loc(interval_days[0])  # find the start index
    end_idx = business_days.get_loc(interval_days[-1])   # find the end index
    total_days = len(interval_days)  # calculate the total number of business days
    
    print(f"{interval_name}:")
    print(f"  Start Index = {start_idx}")
    print(f"  End Index = {end_idx}")
    print(f"  Total Business Days = {total_days}\n")


Total business days: 2782

Training:
  Start Index = 0
  End Index = 1303
  Total Business Days = 1304

Validation:
  Start Index = 1304
  End Index = 2086
  Total Business Days = 783

Test:
  Start Index = 2087
  End Index = 2781
  Total Business Days = 695

